In [2]:
import pandas as pd

In [83]:
earthquakes = pd.read_csv("database.csv")

In [84]:
earthquakes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23412 entries, 0 to 23411
Data columns (total 21 columns):
Date                          23412 non-null object
Time                          23412 non-null object
Latitude                      23412 non-null float64
Longitude                     23412 non-null float64
Type                          23412 non-null object
Depth                         23412 non-null float64
Depth Error                   4461 non-null float64
Depth Seismic Stations        7097 non-null float64
Magnitude                     23412 non-null float64
Magnitude Type                23409 non-null object
Magnitude Error               327 non-null float64
Magnitude Seismic Stations    2564 non-null float64
Azimuthal Gap                 7299 non-null float64
Horizontal Distance           1604 non-null float64
Horizontal Error              1156 non-null float64
Root Mean Square              17352 non-null float64
ID                            23412 non-null object
Sou

### Droping columns which have many missing values

In [85]:
dt = earthquakes.drop(['Depth Error','Depth Seismic Stations','Depth Seismic Stations','Magnitude Error',
                  'Magnitude Seismic Stations','Horizontal Distance','Horizontal Error','Azimuthal Gap','Date',
                      'Time'],axis=1)

In [86]:
dt.shape

(23412, 12)

In [87]:
dt.corr()

,Latitude,Longitude,Depth,Magnitude,Root Mean Square
Latitude,1.000000,0.203546,-0.081020,0.034987,-0.214762
Longitude,0.203546,1.000000,-0.085861,0.038579,-0.028061
Depth,-0.081020,-0.085861,1.000000,0.023457,-0.134002
Magnitude,0.034987,0.038579,0.023457,1.000000,0.075865
Root Mean Square,-0.214762,-0.028061,-0.134002,0.075865,1.000000


In [88]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23412 entries, 0 to 23411
Data columns (total 12 columns):
Latitude            23412 non-null float64
Longitude           23412 non-null float64
Type                23412 non-null object
Depth               23412 non-null float64
Magnitude           23412 non-null float64
Magnitude Type      23409 non-null object
Root Mean Square    17352 non-null float64
ID                  23412 non-null object
Source              23412 non-null object
Location Source     23412 non-null object
Magnitude Source    23412 non-null object
Status              23412 non-null object
dtypes: float64(5), object(7)
memory usage: 2.1+ MB


### Converting my categorical variables to numerical

In [98]:
df = pd.concat([
        dt.select_dtypes([], ['object']),
        dt.select_dtypes(['object']).apply(pd.Series.astype, dtype='category')
        ], axis=1).reindex(dt.columns, axis=1)


In [99]:
cat_columns = df.select_dtypes(['category']).columns

In [100]:
df[cat_columns] = df[cat_columns].apply(lambda x: x.cat.codes)

In [101]:
target = df[['Magnitude']].copy()
X = df.drop('Magnitude',axis=1)

In [102]:
X.head()

,Latitude,Longitude,Type,Depth,Magnitude Type,Root Mean Square,Source,Location Source,Magnitude Source,Status
0,19.246,145.616,0,131.6,5,NaN,4,20,11,0
1,1.863,127.352,0,80.0,5,NaN,4,20,11,0
2,-20.579,-173.972,0,20.0,5,NaN,4,20,11,0
3,-59.076,-23.557,0,15.0,5,NaN,4,20,11,0
4,11.938,126.427,0,15.0,5,NaN,4,20,11,0


In [111]:
X[['Root Mean Square']] = X[['Root Mean Square']].fillna(X[['Root Mean Square']].mean())

In [112]:
from sklearn.model_selection import train_test_split

In [113]:
X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.33, random_state=42)

In [114]:
from sklearn.tree import DecisionTreeRegressor

In [130]:
tree = DecisionTreeRegressor(max_depth=5)

In [131]:
tree.fit(X_train,y_train)

DecisionTreeRegressor(criterion='mse', max_depth=5, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [132]:
y_pred = tree.predict(X_test)

In [133]:
from sklearn.metrics import r2_score
print(tree.score(X_train,y_train))
print(tree.score(X_test, y_test))

0.11240932370398603
0.10531808224689088
